In [ ]:
import tensorflow as tf

import os
import time
import numpy as np
import pandas as pd
from IPython import display

from scipy.stats import spearmanr

In [ ]:
"""
Loading one-hot encoded beta-lactamase and IF1 datasets and spliting into training and test sets.
"""

batch_size=64
split = 0.95


BL_datas= np.load('BL_train_data.npy')
np.random.shuffle(BL_datas)
train_data = BL_datas[ : int(BL_datas.shape[0]*split)]
test_data = BL_datas[int(BL_datas.shape[0]*split): ]
BL_train_data = tf.data.Dataset.from_tensor_slices(train_data).shuffle(len(train_data)).batch(batch_size)
BL_test_data = tf.data.Dataset.from_tensor_slices(test_data).shuffle(len(test_data)).batch(batch_size)


BL_mutation_data = np.load('BL_test_data.npy')
BL_mutation_values = np.load('BL_target_values.npy')

In [ ]:
batch_size = 64
split = 0.95


IF1_datas= np.load('IF1_train_data.npy')
np.random.shuffle(IF1_datas)
train_data = IF1_datas[ : int(IF1_datas.shape[0]*split)]
test_data = IF1_datas[int(IF1_datas.shape[0]*split): ]
IF1_train_data = tf.data.Dataset.from_tensor_slices(train_data).shuffle(len(train_data)).batch(batch_size)
IF1_test_data = tf.data.Dataset.from_tensor_slices(test_data).shuffle(len(test_data)).batch(batch_size)


IF1_mutation_data = np.load('IF1_test_data.npy')
IF1_mutation_values = np.load('IF1_target_values.npy')

In [ ]:
"""
Function that makes sure only 20 canonical residues are used in peptide sequences.
"""
def cleanup(ListofSeqs):
  cleaned_up=[]
  
  dash=np.zeros(20)
  
  Z=np.zeros(20)
  Z[8]=0.5
  Z[10]=0.5
  
  B=np.zeros(20)
  B[7]=0.5
  B[9]=0.5

  X=np.ones(20)
  X=X/20

  for seq in ListofSeqs:
    cleanedSeq=[]
    for alphabet in seq:
      if alphabet[0]==1:
        cleanedSeq.append(dash)
      elif alphabet[1]==1:
        cleanedSeq.append(Z)
      elif alphabet[2]==1:
        cleanedSeq.append(B)
      elif alphabet[23]==1:
        cleanedSeq.append(X)
      else:
        cleanedSeq.append(alphabet[3:23])
    cleaned_up.append(cleanedSeq)

  return np.array(cleaned_up)

In [ ]:
"""
Alternative version where sequences containing Z, B, X are simply discarded.
"""
def cleanupByRemoval(ListofSeqs):
  cleaned_up=[]

  for seq in ListofSeqs:
    cleanedSeq=[]
    check=True
    for alphabet in seq:
      if alphabet[0]==1:
        cleanedSeq.append(np.zeros(20))
      elif alphabet[1]==1:
        check=False
        break
      elif alphabet[2]==1:
        check=False
        break
      elif alphabet[23]==1:
        check=False
        break
      else:
        cleanedSeq.append(alphabet[3:23])
    
    if check:
      cleaned_up.append(cleanedSeq)

  return np.array(cleaned_up)


In [ ]:
print(BL_datas.shape)
BL_datas=cleanupByRemoval(BL_datas)
print(BL_datas.shape)

(8403, 253, 24)
(8355, 253, 20)


In [ ]:
split=0.95
np.random.shuffle(BL_datas)
train_data = BL_datas[ : int(BL_datas.shape[0]*split)]
test_data = BL_datas[int(BL_datas.shape[0]*split): ]
BL_train_data = tf.data.Dataset.from_tensor_slices(train_data).shuffle(len(train_data)).batch(batch_size)
BL_test_data = tf.data.Dataset.from_tensor_slices(test_data).shuffle(len(test_data)).batch(batch_size)

In [ ]:
print(BL_mutation_data.shape)
BL_mutation_data = cleanup(BL_mutation_data)
print(BL_mutation_data.shape)

(4610, 253, 24)
(4610, 253, 20)


In [ ]:
from operator import concat
"""
VAE model consists of inference net and generative net, and latent variables of 
given dimension connects the two parts.
This model is a doubly variational model where the generative net is also a variational network
in addition to the variational latent dimension sampling.
The architecture of this model is from this paper:
Riesselman, A.J., Ingraham, J.B. & Marks, D.S. Deep generative models of genetic 
variation capture the effects of mutations. Nat Methods 15, 816–822 (2018).
"""

class VAEsvi(tf.keras.Model):
  def __init__(self, latent_dim):
    super(VAEsvi, self).__init__()
    self.latent_dim = latent_dim
    self.inference_net = tf.keras.Sequential(
      [
          tf.keras.layers.InputLayer(input_shape=(253, 20)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(1500, activation='relu'),
          tf.keras.layers.Dense(1500, activation='relu'),
          tf.keras.layers.Dense(latent_dim + latent_dim),
      ]
    )

    create_weight = lambda dim_input, dim_output: tf.random.normal(
        [dim_input, dim_output],
        mean = 0.0,
        stddev = tf.math.sqrt(2.0 / (dim_input + dim_output))
        )

    create_bias = lambda dim_output:  0.1 * np.ones(dim_output)

    self.logsig_init = -5

    create_weight_logsig = lambda dim_input, dim_output: self.logsig_init * np.ones((dim_input, dim_output))
    
    create_bias_logsig = lambda dim_output: self.logsig_init * np.ones(dim_output)

    self.W1 = tf.Variable(create_weight(latent_dim, 100), trainable=True)
    self.W1_logsig = tf.Variable(create_weight_logsig(latent_dim, 100), dtype='float32', trainable=True)
    self.B1 = tf.Variable(create_bias(100), dtype='float32', trainable=True)
    self.B1_logsig = tf.Variable(create_bias_logsig(100), dtype='float32', trainable=True)

    self.W2 = tf.Variable(create_weight(100, 2000), trainable=True)
    self.W2_logsig = tf.Variable(create_weight_logsig(100, 2000), dtype='float32', trainable=True)
    self.B2 = tf.Variable(create_bias(2000), dtype='float32', trainable=True)
    self.B2_logsig = tf.Variable(create_bias_logsig(2000), dtype='float32', trainable=True)

    self.W3 = tf.Variable(create_weight(2000, 253*20), trainable=True)
    self.W3_logsig = tf.Variable(create_weight_logsig(2000, 253*20), dtype='float32', trainable=True)
    self.B3 = tf.Variable(create_bias(253*20), dtype='float32', trainable=True)
    self.B3_logsig = tf.Variable(create_bias_logsig(253*20), dtype='float32', trainable=True)

  def encode(self, x):
    mean, logvar = tf.split(self.inference_net(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def reparameterize2(self, mean, logsig):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logsig) + mean

  def decode(self, z):
    w1 = self.reparameterize2(self.W1,self.W1_logsig)
    w2 = self.reparameterize2(self.W2,self.W2_logsig)
    w3 = self.reparameterize2(self.W3,self.W3_logsig)
    b1 = self.reparameterize2(self.B1,self.B1_logsig)
    b2 = self.reparameterize2(self.B2,self.B2_logsig)
    b3 = self.reparameterize2(self.B3,self.B3_logsig)
    
    dense1 = tf.nn.relu(tf.matmul(z, w1) + b1)
    dense2 = tf.nn.sigmoid(tf.matmul(dense1, w2) + b2)
    logits = tf.matmul(dense2, w3) + b3
    return tf.reshape(logits, shape=[logits.shape[0],253,20])
  
  def gen_kld_params(self):
    KLD_params = 0.0
    for i, j in [(self.W1, self.W1_logsig),
                 (self.W2, self.W2_logsig),
                 (self.W3, self.W3_logsig),
                 (self.B1, self.B1_logsig),
                 (self.B2, self.B2_logsig),
                 (self.B3, self.B3_logsig)]:
        mu = tf.reshape(i, [-1])
        log_sigma = tf.reshape(j, [-1])
        KLD_params += tf.reduce_sum(-self.KLD_diag_gaussians(mu, log_sigma, 0.0, 1.0), -1)

    return KLD_params

  def KLD_diag_gaussians(self, mu, log_sigma, prior_mu, prior_log_sigma):
    return prior_log_sigma - log_sigma + 0.5 * (tf.math.exp(2. * log_sigma) \
            + tf.math.square(mu - prior_mu)) * tf.math.exp(-2. * prior_log_sigma) - 0.5


In [ ]:
latent_dim=30
model = VAEsvi(latent_dim)
model.inference_net.summary()
len(model.trainable_variables)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 5060)              0         
                                                                 
 dense (Dense)               (None, 1500)              7591500   
                                                                 
 dense_1 (Dense)             (None, 1500)              2251500   
                                                                 
 dense_2 (Dense)             (None, 60)                90060     
                                                                 
Total params: 9,933,060
Trainable params: 9,933,060
Non-trainable params: 0
_________________________________________________________________


18

In [ ]:
"""
log_normal_pdf function is used to easy calculation of KL divergence, and compute_loss
function calculates ELBO.
Given sequence (x) is used to estimate q(z|x) distribution and reprametrization trick 
is utilized to randomly sample a z value.
Then, x is chosen from p(x|z) distribution and compared with input protein sequence. 
Softmax cross entropy is used to calculate errors.

"""

optimizer = tf.keras.optimizers.Adam(1e-4)

def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)

@tf.function
def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)

  cross_ent = tf.nn.softmax_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=1)
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -(tf.reduce_sum(logpx_z + logpz - logqz_x) + model.gen_kld_params())/x.shape[0]

@tf.function
def compute_apply_gradients(model, x, optimizer):
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
"""
Training the model.
"""

epochs = 100000
tot_time=0
prevelbo=-1000000000
count=0
seedcount=0

for epoch in range(1, epochs + 1):
  np.random.seed(seedcount)
  start_time = time.time()
  for train_x in BL_train_data:
    compute_apply_gradients(model, train_x, optimizer)

  end_time = time.time()
  tot_time+=(end_time-start_time)


  if epoch % 1 == 0:
    loss = tf.keras.metrics.Mean()
    for test_x in BL_test_data:
      loss(compute_loss(model, test_x))
    elbo = -loss.result()
    if elbo <= prevelbo:
      count+=1
    else:
      prevelbo=elbo
      count=0
    display.clear_output(wait=False)
    print('Epoch: {}, Test set ELBO: {}, '
          'time elapse for current epoch {} sec, Total elapsed time {} sec'.format(epoch,
                                                    elbo,
                                                    end_time - start_time, tot_time))
    if epoch % 100 ==0:
      model.save_weights('./checkpointsforever4/my_checkpointforever4')


  seedcount+=1



In [ ]:
WTseq=np.array(list(BL_mutation_data[0]))
WTseq[0]=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]

In [ ]:
@tf.function
def compute_p(model, x, iteration):
  mean, logvar = model.encode(x)
  KLD_latent = 0.5 * tf.reduce_sum(1.0 + logvar - mean**2.0 - tf.exp(logvar), axis=1)

  tileFactor = tf.constant([iteration,1])
  k1 = tf.tile(mean, tileFactor)
  k2 = tf.tile(logvar, tileFactor)



  z = model.reparameterize(k1, k2)
  x_logit = model.decode(z)
  cross_ent = tf.nn.softmax_cross_entropy_with_logits(logits=x_logit, labels=tf.tile(x, tf.constant([iteration,1,1])))
  logpx_z = -tf.reduce_sum(cross_ent, axis=1)
  temp = tf.reshape(logpx_z, [iteration, x.shape[0]])
  logpx_z_Sum = tf.reduce_sum(temp, axis=0)
  
  return KLD_latent + logpx_z_Sum/iteration

In [ ]:
a=[]
tmp=0
Iteration = 100

logpWT = -compute_p(model, tf.constant([WTseq]), Iteration)
logpM= -compute_p(model, BL_mutation_data, Iteration)
a= (logpM-logpWT).numpy()

rho, p = spearmanr(a, BL_mutation_values)
print("Spearman Correlation: ", rho, "\nP Val:", p)